In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.preprocessing import StandardScaler
import joblib

In [3]:
df = pd.read_csv('../../data/training_data.csv', sep=',')
df.drop(columns=['Rating Average'],inplace=True)
df

,Min Players,Max Players,Play Time,Min Age,Users Rated,Complexity Average,Owned Users,Abstract Games,Children's Games,Customizable Games,...,Wargames,Social Interaction,Strategy and Planning,Conflict Resolution,Exploration,Randomness and Luck,Resource Management,Puzzle Solving,Miscellaneous / Other,Collaboration
0,1,4,120.000000,14,42055,3.8604,68323.0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
1,2,4,60.000000,13,41643,2.8405,65294.0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
2,2,4,120.000000,14,19217,3.9129,28785.0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,1,5,120.000000,12,64864,3.2406,87099.0,0,0,0,...,0,0,1,1,0,0,1,1,0,0
4,3,6,431.858995,14,13468,4.2219,16831.0,0,0,0,...,0,1,1,1,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20315,2,2,30.000000,4,1340,1.0000,427.0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
20316,2,99,60.000000,5,2154,1.0455,1533.0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
20317,2,4,30.000000,3,4006,1.0779,5788.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
20318,2,6,30.000000,3,3783,1.0201,4400.0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [5]:
X = df.drop(columns=['Complexity Average'])
y = df['Complexity Average']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline_gbr = Pipeline([
    ('scaler', StandardScaler()), 
    ('gbr', GradientBoostingRegressor(random_state=42))  
])

In [6]:
param_space_gbr = {
    'gbr__n_estimators': (50, 500),
    'gbr__learning_rate': (0.001, 0.2),
    'gbr__max_depth': (3, 10),
    'gbr__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'gbr__min_samples_split': (2, 20),
    'gbr__min_samples_leaf': (1, 10),
    'gbr__max_features': ['sqrt', 'log2'],
    'gbr__alpha': (0.01, 0.1)
}

grid_search = GridSearchCV(pipeline_gbr, param_space_gbr, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f"Best hyperparameters: {best_params}")

Fitting 5 folds for each of 768 candidates, totalling 3840 fits
Best hyperparameters: {'gbr__alpha': 0.01, 'gbr__learning_rate': 0.2, 'gbr__max_depth': 3, 'gbr__max_features': 'sqrt', 'gbr__min_samples_leaf': 10, 'gbr__min_samples_split': 2, 'gbr__n_estimators': 500, 'gbr__subsample': 1.0}


In [7]:
y_pred = grid_search.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R² Score: {r2}")

Mean Squared Error: 0.2265134301883197
Mean Absolute Error: 0.3335946346254992
R² Score: 0.6801652758665828


In [8]:
joblib.dump(grid_search.best_estimator_, '../../models/complexity_gradient.pkl')

['../../models/complexity_gradient.pkl']